In [ ]:
import numpy as np

from source.leap.dynamixel.driver import DynamixelDriver

servo_ids = list(range(16))
driver = DynamixelDriver(
    servo_ids=servo_ids,
    port="/dev/cu.usbserial-FTA2U4SR",
    baud_rate=4_000_000,
    reading_interval=1,
)
driver.set_operation_mode(5)
driver.set_position_p_gain([450, 600, 600, 600, 450, 600, 600, 600, 450, 600, 600, 600, 450, 600, 600, 600])
driver.set_position_d_gain([150, 200, 200, 200, 150, 200, 200, 200, 150, 200, 200, 200, 150, 200, 200, 200])
driver.set_goal_current([350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350, 350])
# driver.set_p_gain([300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300, 300])
# driver.set_p_gain([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100])
# driver.set_velocity_p_gain([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100])

driver.set_torque_mode(enable=True)
mapping = np.array([0, 4, 8, 12, 2, 6, 10, 14, 3, 7, 11, 15, 1, 5, 9, 13])

driver.set_joint_positions(np.zeros(16) + np.pi)

Exception in thread Thread-4 (_reading_loop):
Traceback (most recent call last):
  File "/Users/jdscript/.local/share/uv/python/cpython-3.11.11-macos-aarch64-none/lib/python3.11/threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "/Users/jdscript/Developer/leap-hand-api/.venv/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
  File "/Users/jdscript/.local/share/uv/python/cpython-3.11.11-macos-aarch64-none/lib/python3.11/threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/jdscript/Developer/leap-hand-api/source/leap/dynamixel/driver.py", line 135, in _reading_loop
    dxl_comm_result = self._group_sync_read.fastSyncRead()
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jdscript/Developer/leap-hand-api/.venv/lib/python3.11/site-packages/dynamixel_sdk/group_sync_read.py", line 177, in fastSyncRead
    result = self.fastSyncReadTxPacket()
          

In [2]:
init_pose = np.array(
    [0.000, 0.500, 0.000, 0.000, -0.750, 1.300, 0.000, 0.750, 1.750, 1.500, 1.750, 1.750, 0.00, 1.0000, 0.0000, 0.00]
)[mapping]
driver.set_joint_positions(init_pose + np.pi)

In [5]:
driver.get_joint_positions()

array([3.14466062, 2.38994207, 4.89033075, 3.14312663, 3.15693246,
       3.14312663, 4.89646667, 3.14159265, 3.12778683, 3.88710732,
       4.90106862, 3.14312663, 3.65240826, 4.43473846, 4.65102975,
       4.14941803])

In [9]:
driver.get_joint_positions()[[0, 12, 4, 8, 1, 13, 5, 9, 2, 14, 6, 10, 3, 15, 7, 11]] - np.pi

array([ 3.06796158e-03,  5.10815602e-01,  1.38058271e-02, -1.38058271e-02,
       -7.50116605e-01,  1.29314580e+00,  1.53398079e-03,  7.45514663e-01,
        1.74873810e+00,  1.50943710e+00,  1.75334004e+00,  1.75794198e+00,
        0.00000000e+00,  1.00629140e+00,  0.00000000e+00,  0.00000000e+00])

In [3]:
target = [2705, 2075, 2960, 2740, 2635, 2027, 2795, 2516, 2804, 2077, 3057, 2411, 3237, 1065, 2471, 2960]
target_angle = np.array(target) / 4096 * 2 * np.pi
driver.set_joint_positions(target_angle)

In [3]:
all_actions = np.load("./all_obs.npz")["obs"]
all_actions = all_actions[:, :, mapping]

In [4]:
import time

for action in all_actions:
    curr = action[0]
    curr += np.pi
    driver.set_joint_positions(curr)
    time.sleep(0.1)

KeyboardInterrupt: 

In [ ]:
SIM_LOWER_LIMITS = [
    -0.3140,
    -0.3490,
    -0.3140,
    -0.3140,
    -1.0470,
    -0.4700,
    -1.0470,
    -1.0470,
    -0.5060,
    -1.2000,
    -0.5060,
    -0.5060,
    -0.3660,
    -1.3400,
    -0.3660,
    -0.3660,
]
SIM_LOWER_LIMITS = np.array(SIM_LOWER_LIMITS)

REAL_LOWER_LIMITS = SIM_LOWER_LIMITS[mapping] + np.pi
REAL_LOWER_LIMITS[:4] = np.pi

In [12]:
driver.set_joint_positions(REAL_LOWER_LIMITS)